In [1]:
"""
@author: sheepb
"""

from selenium import webdriver
import pandas as pd
import numpy as np
from datetime import date, timedelta
import re 

driver = webdriver.Chrome('/usr/bin/chromedriver')
driver.get('https://www.worldometers.info/coronavirus/')

# --- Get click on navigator Yesterday --- 

# scrolling on page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight*1/6);")

# find tag and get click
nav_yesterday = driver.find_element_by_id('nav-yesterday2-tab')
nav_yesterday.click()


# --- Get data table ---

table_tag = driver.find_element_by_id("main_table_countries_yesterday2")

colnames_tag = table_tag.find_element_by_tag_name('thead').find_elements_by_tag_name('th')

columns = []
for col in colnames_tag:
    columns.append(col.text)
    
tbody_tag = table_tag.find_element_by_tag_name('tbody')     
lines = tbody_tag.find_elements_by_tag_name('tr')

data = []

for line in lines:
    if line.text == '':
        continue
        
    row = []
    for l in line.find_elements_by_tag_name('td'):
        row.append(l.text)
    data.append(row)

# PREPROCESSING
df = pd.DataFrame(np.array(data), columns=columns)
temp_df = pd.read_csv('temp_df.csv')

df['Continent'] = temp_df['Continent']
df.drop(['#', ''], axis=1, inplace=True)
df.drop(0, axis=0, inplace=True)
df.reset_index()

dict_rename = {column: column.replace('\n', '') for column in df.columns}
df.rename(columns=dict_rename,inplace=True) 

cate_cols = ['Country,Other','Continent']
numberic_cols = df.columns.difference(cate_cols)
df[numberic_cols] = df[numberic_cols].apply\
    (lambda x: x.str.replace(re.compile(r'[+.,]'), '').replace('',np.nan)).fillna('0').astype('int64')

df[cate_cols] = df[cate_cols].fillna('Ship')

# TO CSV
today = date.today()
date = today - timedelta(2)
df.to_csv('Data_get_by_twodayagoCrawl/%s.csv'% str(date))